<a href="https://colab.research.google.com/github/parisazeynaly/explainable-toxic-comment-detection/blob/main/Toxic_Comment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
test= pd.read_csv("test.csv")
train= pd.read_csv("train.csv")

In [ ]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. نمایش اولیه
display(train.head())
display(test.head())

print("Train Info:")
train.info()
print("\nTest Info:")
test.info()

# 2. خلاصه آماری
print("\nTrain Describe:")
display(train.describe())

# 3. مقادیر گمشده
print("\nMissing values in train:\n", train.isnull().sum())
print("\nMissing values in test:\n", test.isnull().sum())

# 4. توزیع تارگت
if "toxic" in train.columns:
    plt.figure(figsize=(6,4))
    sns.countplot(x="toxic", data=train)
    plt.title("Target Distribution (toxic vs non-toxic)")
    plt.show()

# 5. توزیع فیچرهای عددی
numeric_cols = train.select_dtypes(include=['int64','float64']).columns
train[numeric_cols].hist(figsize=(12,8), bins=30)
plt.suptitle("Numeric Features Distribution")
plt.show()


In [ ]:
import pandas as pd

# --- حذف مقادیر گمشده ---
print("Before dropna:", train.shape)
train = train.dropna(subset=["comment_text"])
print("After dropna:", train.shape)

print("Before dropna test:", test.shape)
test = test.dropna(subset=["comment_text"])
print("After dropna test:", test.shape)

# --- حذف ردیف‌های تکراری ---
print("Before drop_duplicates:", train.shape)
train = train.drop_duplicates(subset=["comment_text"])
print("After drop_duplicates:", train.shape)

# --- پاک‌سازی اولیه متن ---
def basic_clean(text):
    if isinstance(text, str):
        return text.strip().replace("\n", " ")
    return text

train["comment_text"] = train["comment_text"].apply(basic_clean)
test["comment_text"] = test["comment_text"].apply(basic_clean)

# بررسی نتیجه
display(train.head())


In [ ]:
# اگر نصب نیست:
!pip install iterative-stratification -q

import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# فرض: train_df را قبلاً لود و پاک‌سازی حداقلی کرده‌ای
# ستون‌های برچسب:
label_cols = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

# فقط ردیف‌هایی که دست‌کم یکی از برچسب‌ها مشخص است (0/1) و متن خالی نیست
df = train.dropna(subset=["comment_text"]).reset_index(drop=True)
# اطمینان از نوع عددی برچسب‌ها
df[label_cols] = df[label_cols].astype(int)

X = df.index.values.reshape(-1, 1)   # ایندکس‌ها را به‌عنوان شناسه نگه می‌داریم
y = df[label_cols].values

# --- مرحله 1: train_temp (90%) + holdout (10%)
msss1 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
train_temp_idx, holdout_idx = next(msss1.split(X, y))

df_train_temp = df.iloc[train_temp_idx].reset_index(drop=True)
df_holdout    = df.iloc[holdout_idx].reset_index(drop=True)

# --- مرحله 2: از train_temp، train (80%) و val (10%) بسازیم => val = 10/90 ≈ 11.11% از train_temp
X_temp = df_train_temp.index.values.reshape(-1, 1)
y_temp = df_train_temp[label_cols].values

msss2 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.1111, random_state=43)
train_idx, val_idx = next(msss2.split(X_temp, y_temp))

df_train = df_train_temp.iloc[train_idx].reset_index(drop=True)
df_val   = df_train_temp.iloc[val_idx].reset_index(drop=True)

print("Final splits:")
print("train:", df_train.shape, "val:", df_val.shape, "holdout:", df_holdout.shape)


In [ ]:
def label_prevalence(frame, label_cols):
    return (frame[label_cols].mean().round(4) * 100).astype(str) + "%"

print("All data:\n", label_prevalence(df, label_cols))
print("\nTrain:\n", label_prevalence(df_train, label_cols))
print("\nVal:\n", label_prevalence(df_val, label_cols))
print("\nHoldout:\n", label_prevalence(df_holdout, label_cols))


In [ ]:
out_dir = "/content/drive/MyDrive/ToxicComments/splits"  # مسیر دلخواهت
import os; os.makedirs(out_dir, exist_ok=True)

df_train.to_csv(f"{out_dir}/train_split.csv", index=False)
df_val.to_csv(f"{out_dir}/val_split.csv", index=False)
df_holdout.to_csv(f"{out_dir}/holdout_split.csv", index=False)


In [ ]:
!pip install transformers -q

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
MAX_LEN = 128   # می‌تونی بعداً 256 یا 512 هم تست کنی


In [ ]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, label_cols):
        self.texts = df["comment_text"].tolist()
        self.labels = df[label_cols].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        labels = torch.tensor(self.labels[idx], dtype=torch.float)

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": labels
        }


In [ ]:
BATCH_SIZE = 32
label_cols = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

train_dataset = ToxicCommentsDataset(df_train, tokenizer, MAX_LEN, label_cols)
val_dataset   = ToxicCommentsDataset(df_val, tokenizer, MAX_LEN, label_cols)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [ ]:
batch = next(iter(train_loader))
print(batch["input_ids"].shape)      # [32, 128]
print(batch["attention_mask"].shape) # [32, 128]
print(batch["labels"].shape)         # [32, 6]


In [ ]:
from transformers import AutoModelForSequenceClassification

# مدل رو لود می‌کنیم
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",   # میشه roberta-base یا distilbert هم امتحان کنی
    num_labels=6,          # چون 6 برچسب داریم
    problem_type="multi_label_classification"
)


In [ ]:
import torch.nn as nn

loss_fn = nn.BCEWithLogitsLoss()


In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

# تعداد کل استپ‌ها
total_steps = len(train_loader) * 3   # مثلاً 3 epoch

# Scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
import time
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EPOCHS = 3

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)

    model.train()
    train_loss = 0
    start_time = time.time()

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    end_time = time.time()
    print(f"Train loss: {avg_train_loss:.4f}")
    print(f"Epoch {epoch+1} training time: {(end_time - start_time):.2f} seconds")

    # --- Validation ---
    model.eval()
    val_loss = 0
    start_time = time.time()

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    end_time = time.time()
    print(f"Validation loss: {avg_val_loss:.4f}")
    print(f"Epoch {epoch+1} validation time: {(end_time - start_time):.2f} seconds")

print("\nTraining complete!")